In [1]:
import numpy as np
import bokeh
from math import pi

from bokeh.embed import components
from scipy import optimize
import pandas as pd
from bokeh.models import Line, ColumnDataSource
from bokeh import palettes

from bokeh.io import output_file, show
# from bokeh.transform import cumsum

from bokeh.models import LogColorMapper
from bokeh.plotting import figure

In [2]:
def linear_func(x, k, b):
    return k * np.asarray(x) + b


def curve_fit(x_data, y_data, function_type='linear'):
    fitted_y = None
    if function_type == 'linear':
        params, params_covariance = optimize.curve_fit(linear_func, x_data, y_data)
        fitted_y = linear_func(x_data, params[0], params[1])
    return fitted_y


def scatter_fit_plot(x, y, y_fit, label, x_name, y_name, toshow):
    x = np.asarray(x)
    y = np.asarray(y)
    y_fit = np.asarray(y_fit)

    radii = (max(max(x),max(y)) + min(min(x),min(y))) * 0.02

    colors = [
        "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
    ]


    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    TOOLTIPS = [
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
        ("Region", "@label"),
    ]

    p = figure(title="Word Choice VS Education Status", plot_height=351,plot_width=352, x_axis_label='Persons Aged 15 Years And Over Completed Year 10 %', y_axis_label=y_name, tools=TOOLS, tooltips=TOOLTIPS,
               background_fill_color='white',
               background_fill_alpha=0.2,
               border_fill_color='white',
               border_fill_alpha=0.5)

    p.title.text_font_size = '14pt'
    p.title.text_color = 'black'
    p.title.align = 'center'
    p.xaxis.major_label_text_color = "black"
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_color = "black"
    p.yaxis.major_label_text_font_size = '12pt'

    p.scatter(x, y, fill_color=colors, fill_alpha=0.6,
              line_color=None, radius=radii,
              hover_color='red')

    line = Line(x='x', y='y', line_color="#666699", line_width=2)
    lines_source = ColumnDataSource(data=dict(x=x, y=y_fit, label=label))
    p.add_glyph(lines_source, line)
    output_file("result1" + '.html')
    if toshow:
        show(p)

    script_map, div_map = components(p)

    return script_map, div_map, p


def pie_chart(x, title, fade_palette=False):
    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    if not fade_palette:
        data['color'] = palettes.Category20c[len(x)]
    else:
        # data['color'] = ['#fffccf','#ffefc3','#ffd4aa','#feb990', '#ff906a'][0:len(x)]
        data['color'] = ['#ffc100', '#ff9a00', '#ff7400', '#ff4d00', '#ff0000'][0:len(x)]

    TOOLS = "hover,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    p = figure(plot_height=375,plot_width=625, title=title, toolbar_location='right',
               tools=TOOLS, tooltips="@country: @value", x_range=(-0.5, 1.0),
               background_fill_color='white',
               background_fill_alpha=0.2,
               border_fill_color='white',
               border_fill_alpha=0.5)

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend='country', source=data)

    p.title.align = 'center'
    p.title.text_font_size = '14pt'
    p.title.text_color = 'black'
    p.legend.label_text_font_size = '9pt'
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None

    # output_file("pie.html")
    output_file(title + '.html')

    show(p)

    script_map, div_map = components(p)

    return script_map, div_map


def plot_map(map_list, title):
    palette = palettes.Viridis6
    # palette = palettes.OrRd
    x_list = map_list[0]
    y_list = map_list[1]
    name_list = map_list[2]
    ratio_list = map_list[3]

    palette.reverse()

    color_mapper = LogColorMapper(palette=palette)

    data = dict(
        x=x_list,
        y=y_list,
        name=name_list,
        rate=ratio_list,
        )

    TOOLS = "pan,wheel_zoom,reset,hover,save"

    p = figure(
            title=title, plot_height=400, plot_width=300, tools=TOOLS,
            x_axis_location=None, y_axis_location=None,
            tooltips=[
                ("Name", "@name"), ("Percentage)", "@rate%"), ("(Long, Lat)", "($x, $y)")
            ])

    p.grid.grid_line_color = None
    p.hover.point_policy = "follow_mouse"

    p.patches('x', 'y', source=data,
                fill_color={'field': 'rate', 'transform': color_mapper},
                fill_alpha=0.7, line_color="white", line_width=0.5)
    show(p)
    # gridplot(p_list, ncols=3)

    # p.patches('x', 'y', source=data,
    #           fill_color='#ff0000',
    #           fill_alpha={'field': 'rate', 'transform': color_mapper}, line_color="white", line_width=0.5)

    # p.patches('x', 'y', source=data)

    # show(p)
    output_file(title + '.html')
    p = None